In [32]:
import pandas as pd
import requests
import wikipediaapi
from bs4 import BeautifulSoup

user_agent = "MSDS452/Assignment1 (https://example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent, 'en')

# Senate

In [6]:
dfs = pd.read_pickle("US_Senate_2024_df_updated_16Jun2024.pickle")p

In [36]:
wiki_wiki.page(pp).links

{'100th United States Congress': 100th United States Congress (id: ??, ns: 0),
 '101st United States Congress': 101st United States Congress (id: ??, ns: 0),
 '102nd United States Congress': 102nd United States Congress (id: ??, ns: 0),
 '103rd United States Congress': 103rd United States Congress (id: ??, ns: 0),
 '104th United States Congress': 104th United States Congress (id: ??, ns: 0),
 '105th United States Congress': 105th United States Congress (id: ??, ns: 0),
 '106th United States Congress': 106th United States Congress (id: ??, ns: 0),
 '107th United States Congress': 107th United States Congress (id: ??, ns: 0),
 '108th United States Congress': 108th United States Congress (id: ??, ns: 0),
 '109th United States Congress': 109th United States Congress (id: ??, ns: 0),
 '110th United States Congress': 110th United States Congress (id: ??, ns: 0),
 '111th United States Congress': 111th United States Congress (id: ??, ns: 0),
 '112th United States Congress': 112th United States

In [39]:
# Exclude tables when scraping links except for the main table (<table class="infobox vcard">)
pp='Chuck Schumer'
p = dfs.loc[dfs['Wiki page (validated)'].notnull(), 'Wiki page (validated)'].tolist()
# pp = p[0]
ppp=sorted([w for w in wiki_wiki.page(pp).links]) 
print('Number of links from wikipediaapi',len(ppp))


pp='Chuck Schumer'
# Fetch the page
page = wiki_wiki.page(pp)

response = requests.get(page.fullurl)
soup = BeautifulSoup(response.content, 'html.parser')
body_content = soup.find('div', class_='mw-body-content')
all_links = body_content.find_all('a', href=True, title=True) # adding title=True seems to limit results to only wikipedia hyperlinks (excludes external links)
print('Number of links from BeautifulSoup',len(all_links))

all_links

Number of links from wikipediaapi 1726
Number of links from BeautifulSoup 3198


[<a href="/wiki/Party_leaders_of_the_United_States_Senate" title="Party leaders of the United States Senate">Senate Majority Leader</a>,
 <a href="/wiki/Incumbent" title="Incumbent">Incumbent</a>,
 <a href="/wiki/Dick_Durbin" title="Dick Durbin">Dick Durbin</a>,
 <a href="/wiki/Mitch_McConnell" title="Mitch McConnell">Mitch McConnell</a>,
 <a class="mw-redirect" href="/wiki/Chair_of_the_Senate_Democratic_Caucus" title="Chair of the Senate Democratic Caucus">Chair of the Senate Democratic Caucus</a>,
 <a href="/wiki/Incumbent" title="Incumbent">Incumbent</a>,
 <a href="/wiki/Mark_Warner" title="Mark Warner">Mark Warner</a>,
 <a href="/wiki/Elizabeth_Warren" title="Elizabeth Warren">Elizabeth Warren</a>,
 <a href="/wiki/Harry_Reid" title="Harry Reid">Harry Reid</a>,
 <a href="/wiki/United_States_Senate" title="United States Senate">United States Senator</a>,
 <a href="/wiki/New_York_(state)" title="New York (state)">New York</a>,
 <a href="/wiki/Incumbent" title="Incumbent">Incumbent</a>

In [48]:
tables = body_content.find_all('table')

table_links = []
# Print the class of each table
for table in tables:
    table_class = table.get('class')
    links = table.find_all('a', href=True)
    # keep the main infobox table
    if not table_class or (table_class and 'infobox' not in table_class):
        table_links+=links

table_links = list(set(table_links))
    

In [49]:
table_links

[<a href="/wiki/Philip_Schuyler" title="Philip Schuyler">Schuyler</a>,
 <a href="/wiki/Jerry_Brown" title="Jerry Brown">Brown (Jerry)</a>,
 <a href="/wiki/Marc_Molinaro" title="Marc Molinaro">Marc Molinaro</a>,
 <a href="/wiki/Casey_Crabtree" title="Casey Crabtree">Crabtree</a>,
 <a href="/wiki/Daniel_Bonham" title="Daniel Bonham">Bonham</a>,
 <a href="/wiki/Harry_S._Truman" title="Harry S. Truman">Truman</a>,
 <a href="/wiki/John_Rizzo_(politician)" title="John Rizzo (politician)">Rizzo</a>,
 <a href="/wiki/Elizabeth_Warren" title="Elizabeth Warren">Warren</a>,
 <a href="/wiki/United_States_Congress" title="United States Congress">United States Congress</a>,
 <a href="/wiki/Template:US_Senate_Rules_chairs" title="Template:US Senate Rules chairs"><abbr title="View this template">v</abbr></a>,
 <a href="/wiki/Edith_Abbott" title="Edith Abbott">Abbott</a>,
 <a href="/wiki/John_Oliver" title="John Oliver">Oliver</a>,
 <a href="/wiki/New_York%27s_9th_congressional_district" title="New York

### Build the hyperlink graph of US Senate candidates' wiki pages

In [ ]:
p = dfs.loc[dfs['Wiki page (validated)'].notnull(), 'Wiki page (validated)'].tolist()
eds=[]
for pp in p:
    try:
        ppp=sorted([w for w in wiki_wiki.page(pp).links]) 
    except Error as e:
#         print(e)
        pass
    ppp=[x for x in ppp if x!=pp]
    ih=set(ppp).intersection(set(p))
    if len(ih)>0:
#         print("%s has %i hyperlinks to webpages: \n %s \n" %(pp,len(ih),", ".join(list(ih))))
        for q in ih:
            eds.append((pp,q))
        print(p.index(pp),len(p))
print(len(eds))

In [ ]:
Gs=nx.DiGraph()
Gs.add_edges_from(eds)
wp_d = dfs.set_index('Wiki page (validated)')['candidate'].to_dict()
Gs=nx.relabel_nodes(Gs, wp_d)
print(len(Gs.nodes()),len(Gs.edges()))

In [ ]:
incumbency_d = dfs.set_index('candidate')['incumbency'].to_dict()
incumbency_d = {key: 'none' if value is None else value for key, value in incumbency_d.items()}
incumbency_d

In [ ]:
party_d = dfs.set_index('candidate')['party'].to_dict()
office_d = dfs.set_index('candidate')['office'].to_dict()
status_d = dfs.set_index('candidate')['status'].to_dict()
state_d = dfs.set_index('candidate')['State'].to_dict()
incumbency_d = dfs.set_index('candidate')['incumbency'].to_dict()
incumbency_d = {key: 'none' if value is None else value for key, value in incumbency_d.items()}
out_degree_d=dict(Gs.out_degree) 
in_degree_d=dict(Gs.in_degree) 

color_d = {}
for n in Gs.nodes():
    if party_d[n]=='Republican':
        color_d[n]='red'
    elif party_d[n]=='Democratic':
        color_d[n]='blue'
    else:
        color_d[n]='lime'

nodedatalist=[(out_degree_d,"out degree"),(in_degree_d,"in degree"),(party_d,"party"),(office_d,"office"),
              (status_d,"status"),(state_d,"state"),(color_d,"color"),(incumbency_d,"incumbency")] 
graph_attributes(Gs,nodedatalist,edgedatalist=None)

ti="The hyperlink graph of the US Senate candidates' wiki pages"
node_sizes=node_sizes_scaling(Gs,a=50,b=10,mode='lin') 
plot = hv_plot_graph(Gs, node_sizes, arrowhead_length=0.015, plot_size=(1000,1000), pos=graphviz_layout(Gs), 
                     nodelabels=0, node_color="color", bundled=0,
                     partition=None, partition_colors=None, edge_color='olive', edge_line_width=1,
                     title=ti, fontsize={'title': '15pt'}, 
                     xoffset=0, yoffset=0, text_font_size='5pt', text_color='midnightblue', bgcolor="white") #.redim(**{"color": None})
dicts_to_remove = ["color"]
# Loop through the dictionaries to remove
for dict_name in dicts_to_remove:
    if dict_name in plot.data.keys():
        del plot.data[dict_name]
plot

In [ ]:
# Save the plots
hv.save(plot, 'senate_plot_updated.html')

# Remove attributes when saving to pickle
for node in Gs.nodes():
    for attr in list(Gs.nodes[node].keys()):
        del Gs.nodes[node][attr]
with open('US_Senate_2024_graph_updated.pickle', 'wb') as f:
    pickle.dump(Gs, f)